In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import re
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

In [ ]:
def tokenize_data(df, max_length, max_target_length):
    # Tokenize inputs (source text)
    inputs = tokenizer(
        list(df["text"]),
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Tokenize targets (summaries) and use them as labels
    targets = tokenizer(
        list(df["summary"]),
        max_length=max_target_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Set the input labels to be the tokenized summaries
    inputs["labels"] = targets["input_ids"]
    return inputs

# Custom Dataset class for tokenized inputs
class TokenizedDataset(Dataset):
    def __init__(self, tokenized_inputs):
        self.input_ids = tokenized_inputs["input_ids"]
        self.attention_mask = tokenized_inputs["attention_mask"]
        self.labels = tokenized_inputs["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }

In [ ]:
# Define the paths to the directories
source_1_documents_path = '/content/drive/MyDrive/bangla_text_summarizer/Source 1/Documents/'
source_1_summaries_path = '/content/drive/MyDrive/bangla_text_summarizer/Source 1/Summaries/'

# Initialize an empty list to store document and summaries data
data = []

# Iterate over each document in the documents directory
for filename in os.listdir(source_1_documents_path):
    if filename.endswith(".txt"):
        # Get the document file path and read its content
        document_path = os.path.join(source_1_documents_path, filename)
        with open(document_path, 'r', encoding='utf-8', errors='ignore') as doc_file:
            document_content = doc_file.read()

        # Derive the base name of the file to find corresponding summaries
        base_filename = os.path.splitext(filename)[0]

        # Construct paths for summary_1 and summary_2
        summary_1_path = os.path.join(source_1_summaries_path, f"{base_filename}_summary_1.txt")
        summary_2_path = os.path.join(source_1_summaries_path, f"{base_filename}_summary_2.txt")

        # Read summary_1 content with UTF-8 encoding and ignore errors
        with open(summary_1_path, 'r', encoding='utf-8', errors='ignore') as sum1_file:
            summary_1_content = sum1_file.read()

        # Read summary_2 content with ISO-8859-1 encoding, ignoring errors
        with open(summary_2_path, 'r', encoding='utf-16', errors='ignore') as sum2_file:
            summary_2_content = sum2_file.read()

        # Clean up summary_2 to remove excess whitespace and non-printable characters
        summary_2_content = re.sub(r'\s+', ' ', summary_2_content).strip()

        # Append document and summaries to the data list
        data.append({"document": document_content, "summary_1": summary_1_content, "summary_2": summary_2_content})

# Create a DataFrame from the data list
df_1 = pd.DataFrame(data)

# Display the resulting DataFrame
df_1.head()

,document,summary_1,summary_2
0,Title:একটি মহল ধর্মের নামে দেশবিরোধী ষড়যন্ত্রে...,জাতীয় সংসদের হুইপ ও আওয়ামী লীগের সাংগঠনিক সম্প...,জাতীয় সংসদের হুইপ ও আওয়ামী লীগের সাংগঠনিক সম্প...
1,Title:আইপিএল এর পর্দা উঠছে আজ \n\nText:\n\nসব ...,সব স্বাভাবিক থাকলে যেই টুর্নামেন্ট শুরুর কথা ছ...,আজ শনিবার (১৯ সেপ্টেম্বর) রাতে পর্দা উঠছে জমজম...
2,Title:ভুটান ও ইসরায়েল যেভাবে কূটনৈতিক সম্পর্ক...,এবার ইসরায়েলের সঙ্গে পূর্ণ কূটনৈতিক সম্পর্ক স...,এবার ইসরায়েলের সঙ্গে পূর্ণ কূটনৈতিক সম্পর্ক স...
3,Title:দিল্লিতে গেরুয়া সমর্থকদের তাণ্ডব! অভিষেক...,দক্ষিণ ২৪ পরগনার শিরাকোলে জে পি নড্ডার কনভয়ে হ...,দক্ষিণ ২৪ পরগনার শিরাকোলে জে পি নড্ডার কনভয়ে হ...
4,Title:১ টাকায় গাড়ি জেতার সুযোগ দারাজে \n\nText...,দেশের বৃহত্তম অনলাইন মার্কেটপ্লেস দারাজ বাংলাদ...,২ নভেম্বর পর্যন্ত দারাজে চলবে “মেইক অ্যা উইশ” ...


In [ ]:
source_2_documents_path = '/content/drive/MyDrive/bangla_text_summarizer/Source 2/text_summarization.csv'
df_2 = pd.read_csv(source_2_documents_path)
df_2.head()

,category,summary,text
0,technology,অ্যাপসে মিলবে ঢাকাসহ তিন জেলা আদালতের তথ্য,"ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙ..."
1,bangladesh,বিজ্ঞান ও প্রকৌশলে মার্কিন সর্বোচ্চ সম্মাননা...,যুক্তরাষ্ট্রে বিজ্ঞান ও প্রকৌশলে পেশা শুরুর প...
2,bangladesh,বিকল্প শিশুখাদ্য গ্রহণে শিশুর মৃত্যু হলে শাস্ত...,বিকল্প শিশুখাদ্য ও বাণিজ্যিকভাবে উত্পাদিত শিশু...
3,bangladesh,ট্রেনে কাটা পড়ে সাবেক সিভিল সার্জনের মৃত্যু,বগুড়ার আদমদীঘির সান্তাহারে গতকাল শনিবার ট্রেন...
4,bangladesh,যাত্রাবাড়ীতে চুলা জ্বালাতে গিয়ে দুই কর্মচারী ...,যাত্রাবাড়ীর একটি রেস্তোরাঁয় গতকাল বুধবার ভোর...


In [ ]:
# Define file paths
source_3_articles_path = '/content/drive/MyDrive/bangla_text_summarizer/Source 3/article.txt'
source_3_summaries_path = '/content/drive/MyDrive/bangla_text_summarizer/Source 3/summary.txt'

# Read the files and split by lines, assuming each line is an article/summary pair
with open(source_3_articles_path, 'r', encoding='utf-8') as file:
    articles = file.readlines()

with open(source_3_summaries_path, 'r', encoding='utf-8') as file:
    summaries = file.readlines()

# Ensure both lists have the same number of entries
if len(articles) != len(summaries):
    raise ValueError("The number of articles and summaries do not match!")

# Create a DataFrame with each article-summary pair in one row
data = {"Article": [article.strip() for article in articles],
        "Summary": [summary.strip() for summary in summaries]}

df_3 = pd.DataFrame(data)

# Display the DataFrame
df_3.head()  # Use df to see the entire DataFrame

,Article,Summary
0,স্ট্যান্ডার্ড চার্টার্ড ব্যাংকের নতুন প্রধান ন...,স্ট্যান্ডার্ড চার্টার্ডের নতুন সিইও আবরার
1,রাজধানী থেকে চামড়া শিল্পগুলো সাভারে স্থানান্তর...,মার্চের মধ্যে সাভারে চামড়া শিল্পের সিইটিপি: মন...
2,দেশীয় শিল্প বিকাশে সরকারের সব ধরনের উদ্যোগ অব্...,ওয়ালটন কারখানায় শিল্পমন্ত্রী
3,একীভূত হতে চলেছে অনলাইনে শ্রেণিবদ্ধ বিজ্ঞাপন স...,একীভূত হচ্ছে এখানেই ডটকমওএলএক্স
4,যাত্রীবাহী একটি বাসে আগুন দেওয়ার আধা ঘণ্টার মধ...,বাসে আগুন: নড়াইলের পৌর মেয়র গ্রেপ্তার


In [ ]:
# Define file paths
source_4_test_path = '/content/drive/MyDrive/bangla_text_summarizer/Source 4/bengali_test.jsonl'
source_4_train_path = '/content/drive/MyDrive/bangla_text_summarizer/Source 4/bengali_train.jsonl'
source_4_val_path = '/content/drive/MyDrive/bangla_text_summarizer/Source 4/bengali_val.jsonl'

# Load each JSONL file into a DataFrame
df_4_test = pd.read_json(source_4_test_path, lines=True)
df_4_train = pd.read_json(source_4_train_path, lines=True)
df_4_val = pd.read_json(source_4_val_path, lines=True)

In [ ]:
df_4_test.head()

,id,url,title,summary,text
0,news-45155508,https://www.bbc.com/bengali/news-45155508,চীন ১০ লাখ উইগুর মুসলিমকে আটকে রেখেছে - জাতিসং...,জাতিসংঘের মানবাধিকার বিষয়ক কমিটি জেনেভাতে চীন...,আটক স্বজনদের আইডি কার্ড হাতে উইগুর নারীদের বিক...
1,news-39127107,https://www.bbc.com/bengali/news-39127107,"কোকা-কোলা, পেপসি নিষিদ্ধ হলো তামিলনাডুতে",ভারতের দক্ষিণাঞ্চলীয় রাজ্য তামিলনাডুর ব্যবসায...,"ভারতের অন্য অঞ্চলেও কোক, পেপসি নিষিদ্ধ করার দা..."
2,news-49141918,https://www.bbc.com/bengali/news-49141918,ডেঙ্গু পরিস্থিতি: হাসপাতালে ভর্তির দরকার না থা...,"স্বাস্থ্য অধিদপ্তরের হিসাব মতে, গত ২৪ ঘণ্টায় ...",সোহরাওয়ার্দী হাসপাতালের চিত্র সাধারণত সেপ্টেম...
3,news-52808250,https://www.bbc.com/bengali/news-52808250,ডোনাল্ড ট্রাম্প: আমেরিকান প্রেসিডেন্টের পোস্ট ...,এই প্রথম সামাজিক মাধ্যমের বিশাল প্রতিষ্ঠান টুই...,ডোনাল্ড ট্রাম্প অভিযোগ করেছেন টু্‌ইটার আমেরিকা...
4,news-49047323,https://www.bbc.com/bengali/news-49047323,বাংলাদেশে বন্যার প্রকোপ: ২৪ ঘণ্টায় ডুবে যাবে ...,বাংলাদেশের বন্যা পূর্বাভাস এবং সতর্কীকরণ কেন্দ...,বন্যার প্রকোপ ছড়িয়ে পড়ছে সারা দেশে। বাংলাদে...


In [ ]:
df_4_train.head()

,id,url,title,summary,text
0,news-54690291,https://www.bbc.com/bengali/news-54690291,দুর্গাপূজার সময়ে যেভাবে শোক পালন করেন 'মহিষাস...,হিন্দু বাঙালীরা যে সময়ে তাদের সবথেকে বড় উৎসব...,দুর্গাপুজায় মহিষাসুর বধ্যে মধ্য দিয়ে অশুভর ও...
1,news-50797621,https://www.bbc.com/bengali/news-50797621,রাশিয়ায় ক্ষমতার ২০ বছর যেভাবে কেটেছে ভ্লাদিম...,ভ্লাদিমির পুতিন তাঁর ক্ষমতায় থাকার ২০ বছর পূর...,গত ২০ বছরে তিনি রাশিয়ার প্রেসিডেন্ট এবং প্রধা...
2,news-46678346,https://www.bbc.com/bengali/news-46678346,সংসদ নির্বাচন: বরিশালে ছয়টি আসন কিন্তু সবার দ...,বাংলাদেশের দক্ষিণাঞ্চলীয় জেলা বরিশাল এখন তুমু...,বরিশাল সদরে চলছে নির্বাচনী প্রচার প্রচারণা। যদ...
3,news-51860832,https://www.bbc.com/bengali/news-51860832,সর্বকালের সর্বশ্রেষ্ঠ বাঙালি: বিবিসি বাংলার জর...,দু'হাজার চার সালে বিবিসি বাংলা একটি 'শ্রোতা জর...,রবীন্দ্রনাথ ঠাকুর রবীন্দ্রনাথ ঠাকুর বাঙালির কা...
4,56310420,https://www.bbc.com/bengali/56310420,৭ই মার্চের ভাষণ: ৫০ বছর আগে রেসকোর্স ময়দানে উ...,'ভাষণ শুরু আগে মাথার উপর দিয়ে বিমান আর হেলিকপ...,আর কুমিল্লা থেকে বাস ভাড়া করে অনেকের সাথে নিজ...


In [ ]:
df_4_val.head()

,id,url,title,summary,text
0,news-45974510,https://www.bbc.com/bengali/news-45974510,জামাল খাসোগজি হত্যা: সৌদি রাজতন্ত্রের সমালোচক ...,২০১৫ থেকে ২০১৭ সাল পর্যন্ত ইউরোপে বসবাসরত তিনজ...,"প্রিন্স তুর্কি, প্রিন্স সুলতান আর প্রিন্স সাইফ..."
1,news-43807587,https://www.bbc.com/bengali/news-43807587,বাংলাদেশসহ নানা দেশে ম্যালেরিয়া বাড়ায় উদ্বেগ,গত ১০ বছরের মধ্যে এই প্রথমবারের মতো বিশ্বে ম্য...,ম্যালেরিয়া ছড়ায় মশাবাহিত একরকম পরজীবী জীবাণ...
2,160814_alchohol_syndrome_foetal_boy_south_africa,https://www.bbc.com/bengali/news/2016/08/16081...,ভ্রুণের মধ্যে অ্যালকোহল সিনড্রোম,ভ্রুণের মধ্যে অ্যালকোহল সিনড্রোম-বিশ্বের মধ্যে...,যেসব মায়েরা গর্ভাবস্থায় মদ্যপান করেন তাদের স...
3,news-40696034,https://www.bbc.com/bengali/news-40696034,চোখে টিয়ারশেলের আঘাত পাওয়া সিদ্দিকুর কি দৃষ্...,পুলিশের ছোড়া টিয়ারশেলের আঘাতে আহত ছাত্র মোহা...,জাতীয় চক্ষুবিজ্ঞান ইন্সটিটিউট ও হাসপাতাল জাতী...
4,news-45233390,https://www.bbc.com/bengali/news-45233390,রাস্তায় সাইনবোর্ড লাগিয়ে বান্ধবীর কাছে ক্ষমা...,কথা ছিল সে আসবে শুক্রবার। তার আগেই এক দফা ঝগড়...,শহর ছেয়ে গেছে এমন বিলবোর্ডে। এসব ভাবতে ভাবতে ...


In [ ]:
# Define the root directories
source_5_bnlpc_dataset1_root = '/content/drive/MyDrive/bangla_text_summarizer/Source 5/BNLPC/Dataset1'
source_5_bnlpc_dataset2_root = '/content/drive/MyDrive/bangla_text_summarizer/Source 5/BNLPC/Dataset2'
source_5_nctb_root = '/content/drive/MyDrive/bangla_text_summarizer/Source 5/NCTB'

# Function to load articles, summaries, normalized documents, and titles
def load_bnlpc_dataset(documents_path, normalize_document_path, normalize_title_path, summaries_path):
    documents = []
    normalized_documents = []
    normalized_titles = []
    summaries = []

    # Get all files (assumes filenames match for each pair)
    doc_files = sorted(os.listdir(documents_path))
    norm_doc_files = sorted(os.listdir(normalize_document_path))
    norm_title_files = sorted(os.listdir(normalize_title_path))
    summary_files = sorted(os.listdir(summaries_path))

    for doc_file, norm_doc_file, norm_title_file, summary_file in zip(doc_files, norm_doc_files, norm_title_files, summary_files):
        # Read the document
        with open(os.path.join(documents_path, doc_file), 'r', encoding='utf-8') as f:
            documents.append(f.read().strip())

        # Read the normalized document
        with open(os.path.join(normalize_document_path, norm_doc_file), 'r', encoding='utf-8') as f:
            normalized_documents.append(f.read().strip())

        # Read the normalized title
        with open(os.path.join(normalize_title_path, norm_title_file), 'r', encoding='utf-8') as f:
            normalized_titles.append(f.read().strip())

        # Read the summary
        with open(os.path.join(summaries_path, summary_file), 'r', encoding='utf-8') as f:
            summaries.append(f.read().strip())

    # Create a DataFrame with the documents, normalized documents, titles, and summaries
    return pd.DataFrame({
        'Document': documents,
        'NormalizeDocument': normalized_documents,
        'NormalizeTitle': normalized_titles,
        'Summary': summaries
    })

# Load BNLPC Dataset1
bnlpc_dataset1_documents_path = os.path.join(source_5_bnlpc_dataset1_root, 'Documents')
bnlpc_dataset1_normalize_document_path = os.path.join(source_5_bnlpc_dataset1_root, 'NormaliseDocument')
bnlpc_dataset1_normalize_title_path = os.path.join(source_5_bnlpc_dataset1_root, 'NormaliseTitle')
bnlpc_dataset1_summaries_path = os.path.join(source_5_bnlpc_dataset1_root, 'Summaries')

df_5_bnlpc_dataset1 = load_bnlpc_dataset(
    bnlpc_dataset1_documents_path,
    bnlpc_dataset1_normalize_document_path,
    bnlpc_dataset1_normalize_title_path,
    bnlpc_dataset1_summaries_path
)

# Load BNLPC Dataset2
bnlpc_dataset2_documents_path = os.path.join(source_5_bnlpc_dataset2_root, 'Documents')
bnlpc_dataset2_normalize_document_path = os.path.join(source_5_bnlpc_dataset2_root, 'NormaliseDocument')
bnlpc_dataset2_normalize_title_path = os.path.join(source_5_bnlpc_dataset2_root, 'NormaliseTitle')
bnlpc_dataset2_summaries_path = os.path.join(source_5_bnlpc_dataset2_root, 'Summaries')

df_5_bnlpc_dataset2 = load_bnlpc_dataset(
    bnlpc_dataset2_documents_path,
    bnlpc_dataset2_normalize_document_path,
    bnlpc_dataset2_normalize_title_path,
    bnlpc_dataset2_summaries_path
)

# Function to load NCTB dataset (just source and summary)
def load_nctb_dataset(source_path, summary_path):
    sources = []
    summaries = []

    source_files = sorted(os.listdir(source_path))
    summary_files = sorted(os.listdir(summary_path))

    for source_file, summary_file in zip(source_files, summary_files):
        # Read the source document
        with open(os.path.join(source_path, source_file), 'r', encoding='utf-8') as f:
            sources.append(f.read().strip())

        # Read the summary
        with open(os.path.join(summary_path, summary_file), 'r', encoding='utf-8') as f:
            summaries.append(f.read().strip())

    # Create a DataFrame with the source and summary
    return pd.DataFrame({
        'Source': sources,
        'Summary': summaries
    })

# Load NCTB dataset
nctb_source_path = os.path.join(source_5_nctb_root, 'Source')
nctb_summary_path = os.path.join(source_5_nctb_root, 'Summary')
df_5_nctb = load_nctb_dataset(nctb_source_path, nctb_summary_path)

In [ ]:
df_5_bnlpc_dataset1.head()

,Document,NormalizeDocument,NormalizeTitle,Summary
0,Title: লতিফ সিদ্দিকী ছয় মাসের জামিন \n\nText: ...,ধর্মীয় অনুভূতিতে আঘাতের অভিযোগে করা আরও ১০ মাম...,লতিফ সিদ্দিকী ছয় মাসের জামিন,উচ্চ আদালতে নির্দোষ প্রমাণিত হওয়ার পর গতকাল ব...
1,Title: \nযে শর্তে অজয়ের ঘরনী হয়েছিলেন কাজল \n\...,জনপ্রিয়তার তুঙ্গে থাকার সময় ১৯৯৯ সালে কাজল বিয়...,যে শর্তে অজয়ের ঘরনী হয়েছিলেন কাজল,উচ্চ আদালতে নির্দোষ প্রমাণিত হওয়ার পর গতকাল ব...
2,Title: \nখালাসের রায়ের ১৩ বছর পর কারামুক্ত হলে...,উচ্চ আদালতে নির্দোষ প্রমাণিত হওয়ার পর গতকাল ব...,খালাসের রায়ের ১৩ বছর পর কারামুক্ত হলেন জবেদ আলী,উচ্চ আদালতে নির্দোষ প্রমাণিত হওয়ার পর গতকাল ব...
3,Title: \nরংপুরে মানহানির মামলায় মাহফুজ আনামের ...,রংপুরে ১০০ কোটি টাকার মানহানির মামলায় ডেইলি স্...,রংপুরে মানহানির মামলায় মাহফুজ আনামের জামিন মঞ্জুর,জনপ্রিয়তার তুঙ্গে থাকার সময় ১৯৯৯ সালে কাজল বিয়...
4,Title: \nসাত মামলায় জামিন মিন্টুর \n\nText: \...,গাড়ি ভাঙচুর ও নাশকতার অভিযোগে পুলিশের করা সাত...,সাত মামলায় জামিন মিন্টুর,জনপ্রিয়তার তুঙ্গে থাকার সময় ১৯৯৯ সালে কাজল বিয়...


In [ ]:
df_5_bnlpc_dataset2.head()

,Document,NormalizeDocument,NormalizeTitle,Summary
0,"Title: সোনারগাঁয়ে মহাসড়কে যুবলীগের সভা, দীর্...",ঢাকা-চট্টগ্রাম মহাসড়কের ওপর মঞ্চ বানিয়ে গতকাল...,"সোনারগাঁয়ে মহাসড়কে যুবলীগের সভা, দীর্ঘ যানজট...",খালেদা জিয়ার বিরুদ্ধে মামলা দিয়ে সরকার বিএনপির...
1,Title: \nট্যাংকে পিষ্ট বিদ্রোহ-প্রচেষ্টা \n\n...,আল্লাহর নামে আর অখণ্ড পাকিস্তান রক্ষার অজুহাতে...,ট্যাংকে পিষ্ট বিদ্রোহ-প্রচেষ্টা,খালেদা জিয়ার বিরুদ্ধে মামলা দিয়ে সরকার বিএনপির...
2,Title: \nসরকার বিএনপির কাউন্সিল ‘ভন্ডুলের ষড়যন...,খালেদা জিয়ার বিরুদ্ধে মামলা দিয়ে সরকার বিএনপির...,সরকার বিএনপির কাউন্সিল ‘ভন্ডুলের ষড়যন্ত্রে লিপ্ত’,"তিনি বলেন, \""সরকার বিএনপির চেয়ারপারসন খালেদা জ..."
3,Title: \nবেআইনি প্লট ফেরতে দায়মুক্তি \n\nText:...,ইংল্যান্ডের প্রাচীন যুগের নৃপতিরা ব্যক্তিবিশেষ...,বেআইনি প্লট ফেরতে দায়মুক্তি,আল্লাহর নামে আর অখণ্ড পাকিস্তান রক্ষার অজুহাতে...
4,Title: \nগুণীজন ও কৃতী শিক্ষার্থীদের সংবর্ধনা ...,সৌদি আরবের রিয়াদে প্রবাসী বাংলাদেশি গুণীজন ও ক...,গুণীজন ও কৃতী শিক্ষার্থীদের সংবর্ধনা,ঠান্ডা মাথায় পাকিস্তানি সৈন্যরা টানা ২৪ ঘণ্টা ...


In [ ]:
df_5_nctb.head()

,Source,Summary
0,মানুষের সুন্দর মুখ দেখে আনন্দিত হয়ো না। স্বভাব...,"বাহ্যিক সৌন্দর্য নয়, স্বভাবের সৌন্দর্যই মানুষক..."
1,"মুখে অনেকেই টাকা তুচ্ছ, অর্থ অনর্থের মূল বলিয়া...",অর্থকে অনর্থের মূল বলা হলেও মূলত এটি অত্যন্ত ম...
2,"আমি ভাগিনীদের কল্যাণ কামনা করি, তাহাদের ধর্মবন...",উপযুক্ত শিক্ষার অভাবে নারী তার মনের স্বাধীনতা ...
3,"আমি ভালোবেসেছি এ জগৎকে, আমি প্রণাম করেছি মহৎকে...",একজন জীবনসাধকের কাছে এ জগতই সত্য। তিনি মানুষের...
4,আমি লাইব্রেরিকে স্কুল কলেজের উপরে স্থান দিই এই...,স্বশিক্ষিত হবার সুযোগ আমাদের প্রচলিত শিক্ষা ব্...


In [ ]:
# Initialize tokenizer (replace with your desired model's tokenizer)
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

# Function to calculate descriptive statistics for token length
def descriptive_statistics_tokens(df, column_name):
    # Calculate length of each entry in terms of tokens
    df[f"{column_name}_token_length"] = df[column_name].apply(lambda x: len(tokenizer.tokenize(x)))

    # Calculate descriptive statistics
    stats = {
        "mean_length": df[f"{column_name}_token_length"].mean(),
        "max_length": df[f"{column_name}_token_length"].max(),
        "min_length": df[f"{column_name}_token_length"].min(),
        "std_length": df[f"{column_name}_token_length"].std()
    }
    return stats

# Calculate statistics for text and summary columns in each dataset
train_text_stats = descriptive_statistics_tokens(df_4_train, "text")
train_summary_stats = descriptive_statistics_tokens(df_4_train, "summary")

val_text_stats = descriptive_statistics_tokens(df_4_val, "text")
val_summary_stats = descriptive_statistics_tokens(df_4_val, "summary")

test_text_stats = descriptive_statistics_tokens(df_4_test, "text")
test_summary_stats = descriptive_statistics_tokens(df_4_test, "summary")

# Display the statistics
print("Training Set - Text (token count):", train_text_stats)
print("Training Set - Summary (token count):", train_summary_stats)

print("\nValidation Set - Text (token count):", val_text_stats)
print("Validation Set - Summary (token count):", val_summary_stats)

print("\nTest Set - Text (token count):", test_text_stats)
print("Test Set - Summary (token count):", test_summary_stats)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting

Training Set - Text (token count): {'mean_length': 1576.5170328313995, 'max_length': 9645, 'min_length': 23, 'std_length': 943.4472687640438}
Training Set - Summary (token count): {'mean_length': 61.15341890891138, 'max_length': 316, 'min_length': 5, 'std_length': 25.42662109193043}

Validation Set - Text (token count): {'mean_length': 1266.4832015810277, 'max_length': 2559, 'min_length': 153, 'std_length': 540.3451517330489}
Validation Set - Summary (token count): {'mean_length': 56.784584980237156, 'max_length': 105, 'min_length': 22, 'std_length': 17.92673957020336}

Test Set - Text (token count): {'mean_length': 1302.6215415019763, 'max_length': 2548, 'min_length': 182, 'std_length': 542.456299479598}
Test Set - Summary (token count): {'mean_length': 57.50691699604743, 'max_length': 105, 'min_length': 21, 'std_length': 17.74996556414175}


In [ ]:
max_length = 512
max_target_length = 100

In [ ]:
model_name = "google/mt5-small"  # Pretrained mT5-small model
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

# Tokenize the datasets with labels
train_inputs = tokenize_data(df_4_train, max_length=512, max_target_length=100)
val_inputs = tokenize_data(df_4_val, max_length=512, max_target_length=100)
test_inputs = tokenize_data(df_4_test, max_length=512, max_target_length=100)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Create datasets from tokenized inputs
train_dataset = TokenizedDataset(train_inputs)
val_dataset = TokenizedDataset(val_inputs)
test_dataset = TokenizedDataset(test_inputs)

In [ ]:
# Define Seq2SeqTrainingArguments for generation tasks
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,  # Enables generation during evaluation
    save_safetensors=False  # Disable saving in safetensors format
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Define Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer  # Pass the tokenizer for decoding during evaluation
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tashfiqul-islam (tashfiqul-islam-north-south-university). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,8.211700,4.409328
2,3.535100,2.026162
3,2.953800,1.701377


TrainOutput(global_step=3039, training_loss=6.998700407077781, metrics={'train_runtime': 2245.1863, 'train_samples_per_second': 10.826, 'train_steps_per_second': 1.354, 'total_flos': 1.285178998652928e+16, 'train_loss': 6.998700407077781, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 1.7013773918151855, 'eval_runtime': 23.1363, 'eval_samples_per_second': 43.741, 'eval_steps_per_second': 5.489, 'epoch': 3.0}


In [ ]:
# Evaluate on the test dataset
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print(eval_results)

{'eval_loss': 1.7323390245437622, 'eval_runtime': 23.5862, 'eval_samples_per_second': 42.906, 'eval_steps_per_second': 5.385, 'epoch': 3.0}


In [ ]:
df_4_test.head()

,id,url,title,summary,text,text_token_length,summary_token_length
0,news-45155508,https://www.bbc.com/bengali/news-45155508,চীন ১০ লাখ উইগুর মুসলিমকে আটকে রেখেছে - জাতিসং...,জাতিসংঘের মানবাধিকার বিষয়ক কমিটি জেনেভাতে চীন...,আটক স্বজনদের আইডি কার্ড হাতে উইগুর নারীদের বিক...,1206,82
1,news-39127107,https://www.bbc.com/bengali/news-39127107,"কোকা-কোলা, পেপসি নিষিদ্ধ হলো তামিলনাডুতে",ভারতের দক্ষিণাঞ্চলীয় রাজ্য তামিলনাডুর ব্যবসায...,"ভারতের অন্য অঞ্চলেও কোক, পেপসি নিষিদ্ধ করার দা...",788,42
2,news-49141918,https://www.bbc.com/bengali/news-49141918,ডেঙ্গু পরিস্থিতি: হাসপাতালে ভর্তির দরকার না থা...,"স্বাস্থ্য অধিদপ্তরের হিসাব মতে, গত ২৪ ঘণ্টায় ...",সোহরাওয়ার্দী হাসপাতালের চিত্র সাধারণত সেপ্টেম...,1350,44
3,news-52808250,https://www.bbc.com/bengali/news-52808250,ডোনাল্ড ট্রাম্প: আমেরিকান প্রেসিডেন্টের পোস্ট ...,এই প্রথম সামাজিক মাধ্যমের বিশাল প্রতিষ্ঠান টুই...,ডোনাল্ড ট্রাম্প অভিযোগ করেছেন টু্‌ইটার আমেরিকা...,1993,57
4,news-49047323,https://www.bbc.com/bengali/news-49047323,বাংলাদেশে বন্যার প্রকোপ: ২৪ ঘণ্টায় ডুবে যাবে ...,বাংলাদেশের বন্যা পূর্বাভাস এবং সতর্কীকরণ কেন্দ...,বন্যার প্রকোপ ছড়িয়ে পড়ছে সারা দেশে। বাংলাদে...,1248,66


In [ ]:
sample_text = df_4_test.iloc[0]['text']
print(type(sample_text))

<class 'str'>


In [ ]:
# Make sure the input_ids tensor is on the same device as the model
device = model.device  # Get the device of the model (e.g., 'cuda:0' or 'cpu')
input_ids = tokenizer.encode(sample_text, return_tensors="pt", max_length=512, truncation=True).to(device)

# Generate summary
outputs = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

<extra_id_0> হিসেবে চীনা প্রেসিডেন্ট শি জিনপিং প্রদেশকে অভিযুক্ত করেছে।


In [ ]:
df_4_test.iloc[0]['summary']

'জাতিসংঘের মানবাধিকার বিষয়ক কমিটি জেনেভাতে চীনের ওপর দুদিনের যে বিশেষ সভা করছে, সেখানে অভিযোগ করা হয়, কট্টরপন্থী সন্দেহে ১০ লাখ চীনা উইগুর মুসলিমকে বিভিন্ন রাজনৈতিক শিবিরে আটকে রাখা হয়েছে।'

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.369700,1.472677
2,2.123700,1.425006
3,2.091400,1.409577


TrainOutput(global_step=3039, training_loss=2.253774700058099, metrics={'train_runtime': 2240.1625, 'train_samples_per_second': 10.85, 'train_steps_per_second': 1.357, 'total_flos': 1.285178998652928e+16, 'train_loss': 2.253774700058099, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 1.409576654434204, 'eval_runtime': 23.0218, 'eval_samples_per_second': 43.958, 'eval_steps_per_second': 5.517, 'epoch': 3.0}


In [ ]:
# Evaluate on the test dataset
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print(eval_results)

{'eval_loss': 1.4490957260131836, 'eval_runtime': 23.1836, 'eval_samples_per_second': 43.651, 'eval_steps_per_second': 5.478, 'epoch': 3.0}


In [ ]:
sample_text = df_4_test.iloc[5]['text']
print(sample_text)

মাওবাদী এলাকায় পুলিশের টহল (ফাইল ফটো) ভারতে যে রাজ্যগুলোতে মাওবাদীদের দাপট বেশি, সেগুলোর মুখ্যমন্ত্রী ও পুলিশ-প্রধানদের সঙ্গে বৈঠকেই তিনি সোমবার এ মন্তব্য করেন। মাওবাদীদের সঙ্গে শান্তি-আলোচনার কথা যে সরকার আদৌ ভাবছে না, সেটাও তিনি স্পষ্ট করে দিয়েছেন। দুসপ্তাহ আগে ছত্তিশগড়ের সুকমায় মাওবাদীদের হামলায় ২৫জন সিআরপিএফ জওয়ান নিহত হওয়ার পরই ভারতের কেন্দ্রীয় সরকার তড়িঘড়ি আজকের এই বৈঠক ডেকেছিল। সেখানে স্বরাষ্ট্রমন্ত্রী রাজনাথ সিং স্পষ্ট করে দেন মাওবাদীদের দমনে সরকারি কৌশলে বিরাট কোনও পরিবর্তন হচ্ছে না। তিনি বলেন, "সিলভার বুলেট দিয়ে এই সমস্যা মিটবে না - অর্থাৎ ঝটিতি কোনও জাদুবলে এর সমাধান মিলবে না, আর এর কোনও শর্টকাটও নেই। স্বল্প, মাঝারি ও দীর্ঘমেয়াদে নানা ধরনের হাতিয়ার ব্যবহার করেই আমাদের এগোতে হবে।" ইংরেজিতে সমাধান শব্দের এস, এ, এম ইত্যাদি সবগুলো বর্ণের জন্য স্মার্ট লিডারশিপ, অ্যাগ্রেসিভ স্ট্র্যাটেজি, মোটিভেশন এজাতীয় নানা শব্দ ব্যবহার করে স্বরাষ্ট্রমন্ত্রী দাবি করেন, এই সমাধান শব্দেই লুকিয়ে আছে মাওবাদীদের জবাব। আরও পড়ুন: নির্বাচনের পালে হাওয়া, নেই আগাম ভোটের ইঙ্গিত কিভাবে সনাক্

In [ ]:
# Make sure the input_ids tensor is on the same device as the model
device = model.device  # Get the device of the model (e.g., 'cuda:0' or 'cpu')
input_ids = tokenizer.encode(sample_text, return_tensors="pt", max_length=512, truncation=True).to(device)

# Generate summary
outputs = model.generate(input_ids, max_length=200, num_beams=7, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

বাংলাদেশে মাওবাদীদের হামলায় ২৫জন সিআরপিএফ জওয়ান নিহত হওয়ার পর ভারতের কেন্দ্রীয় সরকার তড়িঘড়ি আজকের এই বৈঠক ডেকেছে।


In [ ]:
df_4_test.iloc[5]['summary']

'ভারতে মাওবাদী বিদ্রোহীদের মোকাবিলার পথ খুঁজতে সরকারের এক উচ্চ পর্যায়ের বৈঠকের পর স্বরাষ্ট্রমন্ত্রী রাজনাথ সিং জানিয়েছেন, এই সঙ্কটের কোনও চটজলদি সমাধান সম্ভব নয়, সর্বাত্মক উন্নয়ন আর আগ্রাসী নীতি বজায় রেখেই মাওবাদীদের কাবু করা সম্ভব।'

In [ ]:
# Ensure the model is on CPU
model.to("cpu")

# Make sure the input_ids tensor is on the same device as the model (CPU in this case)
device = model.device  # This should now be 'cpu'
input_ids = tokenizer.encode(sample_text, return_tensors="pt", max_length=1024, truncation=True).to(device)

# Generate summary
outputs = model.generate(input_ids, max_length=200, num_beams=7, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

বাংলাদেশে মাওবাদী বিদ্রোহ দমনের সহজ সমাধান নেই।


In [ ]:
save_path = '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/'

# Make all model parameters contiguous
for param in model.parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

# Save the model and tokenizer
model.save_pretrained(f"{save_path}fine_tuned_mt5_summarization")
tokenizer.save_pretrained(f"{save_path}fine_tuned_mt5_summarization")

('/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization/tokenizer_config.json',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization/special_tokens_map.json',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization/spiece.model',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization/added_tokens.json',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization/tokenizer.json')

In [ ]:
max_length = 512
max_target_length = 200

In [ ]:
# Define the path to your saved model and tokenizer
load_path = '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization_v3'

# Load the tokenizer
tokenizer = MT5Tokenizer.from_pretrained(load_path)

# Load the model
model = MT5ForConditionalGeneration.from_pretrained(load_path)

# Tokenize the datasets with labels
train_inputs = tokenize_data(df_4_train, max_length, max_target_length)
val_inputs = tokenize_data(df_4_val,  max_length, max_target_length)
test_inputs = tokenize_data(df_4_test,  max_length, max_target_length)

In [ ]:
# Create datasets from tokenized inputs
train_dataset = TokenizedDataset(train_inputs)
val_dataset = TokenizedDataset(val_inputs)
test_dataset = TokenizedDataset(test_inputs)

In [ ]:
# Define Seq2SeqTrainingArguments for generation tasks
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,  # Enables generation during evaluation
    save_safetensors=False  # Disable saving in safetensors format
)

In [ ]:
# Define Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer  # Pass the tokenizer for decoding during evaluation
)

In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.046200,0.692979
2,1.015400,0.683604
3,1.027700,0.676918
4,0.988000,0.672858
5,0.994400,0.671896


TrainOutput(global_step=5065, training_loss=1.0143020765449546, metrics={'train_runtime': 5081.3084, 'train_samples_per_second': 7.972, 'train_steps_per_second': 0.997, 'total_flos': 2.14196499775488e+16, 'train_loss': 1.0143020765449546, 'epoch': 5.0})

In [ ]:
sample_text = df_4_test.iloc[5]['text']
print(sample_text)

মাওবাদী এলাকায় পুলিশের টহল (ফাইল ফটো) ভারতে যে রাজ্যগুলোতে মাওবাদীদের দাপট বেশি, সেগুলোর মুখ্যমন্ত্রী ও পুলিশ-প্রধানদের সঙ্গে বৈঠকেই তিনি সোমবার এ মন্তব্য করেন। মাওবাদীদের সঙ্গে শান্তি-আলোচনার কথা যে সরকার আদৌ ভাবছে না, সেটাও তিনি স্পষ্ট করে দিয়েছেন। দুসপ্তাহ আগে ছত্তিশগড়ের সুকমায় মাওবাদীদের হামলায় ২৫জন সিআরপিএফ জওয়ান নিহত হওয়ার পরই ভারতের কেন্দ্রীয় সরকার তড়িঘড়ি আজকের এই বৈঠক ডেকেছিল। সেখানে স্বরাষ্ট্রমন্ত্রী রাজনাথ সিং স্পষ্ট করে দেন মাওবাদীদের দমনে সরকারি কৌশলে বিরাট কোনও পরিবর্তন হচ্ছে না। তিনি বলেন, "সিলভার বুলেট দিয়ে এই সমস্যা মিটবে না - অর্থাৎ ঝটিতি কোনও জাদুবলে এর সমাধান মিলবে না, আর এর কোনও শর্টকাটও নেই। স্বল্প, মাঝারি ও দীর্ঘমেয়াদে নানা ধরনের হাতিয়ার ব্যবহার করেই আমাদের এগোতে হবে।" ইংরেজিতে সমাধান শব্দের এস, এ, এম ইত্যাদি সবগুলো বর্ণের জন্য স্মার্ট লিডারশিপ, অ্যাগ্রেসিভ স্ট্র্যাটেজি, মোটিভেশন এজাতীয় নানা শব্দ ব্যবহার করে স্বরাষ্ট্রমন্ত্রী দাবি করেন, এই সমাধান শব্দেই লুকিয়ে আছে মাওবাদীদের জবাব। আরও পড়ুন: নির্বাচনের পালে হাওয়া, নেই আগাম ভোটের ইঙ্গিত কিভাবে সনাক্

In [ ]:
# Make sure the input_ids tensor is on the same device as the model
device = model.device  # Get the device of the model (e.g., 'cuda:0' or 'cpu')
input_ids = tokenizer.encode(sample_text, return_tensors="pt", max_length=512, truncation=True).to(device)

# Generate summary
outputs = model.generate(input_ids, max_length=200, num_beams=7, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

বাংলাদেশে মাওবাদীদের হামলায় ২৫জন সিআরপিএফ জওয়ান নিহত হওয়ার পর ভারতের কেন্দ্রীয় সরকার তড়িঘড়ি আজকের বৈঠক ডেকেছে।


In [ ]:
df_4_test.iloc[5]['summary']

'ভারতে মাওবাদী বিদ্রোহীদের মোকাবিলার পথ খুঁজতে সরকারের এক উচ্চ পর্যায়ের বৈঠকের পর স্বরাষ্ট্রমন্ত্রী রাজনাথ সিং জানিয়েছেন, এই সঙ্কটের কোনও চটজলদি সমাধান সম্ভব নয়, সর্বাত্মক উন্নয়ন আর আগ্রাসী নীতি বজায় রেখেই মাওবাদীদের কাবু করা সম্ভব।'

In [ ]:
sample_text = """‘ইকোনমিস্ট’ লিখেছে, অ্যান্টিবডির চার মাস স্থায়ী হওয়ার খবরটি দুই কারণে আনন্দের। অ্যান্টিবডি যত দিন পর্যন্ত শরীরে টিকবে, তত দিন সংক্রমণ থেকে সুরক্ষিত থাকা সম্ভব। অর্থাৎ, এমন এক টিকার প্রয়োজন হবে, যা অ্যান্টিবডির উত্পাদনকে প্ররোচিত করতে পারে এবং দীর্ঘস্থায়ী সুরক্ষা দিতে পারে। এগুলো খুঁজে বের করাও সহজ। এটি আভাস দেয়, ব্যাপক হারে অ্যান্টিবডি শনাক্তকরণ ফলাফল মোটামুটি নির্ভুল হওয়া উচিত। দ্বিতীয় আরেকটি গবেষণার নেতৃত্ব দিয়েছেন যুক্তরাজ্যের মেডিকেল রিসার্চ কাউন্সিলের (এমআরসি) ইমিউনোলজিস্ট তাও দং। তিনি টি-সেল শনাক্তকরণে কাজ করেছেন। টি-সেল শনাক্তকরণের প্রক্রিয়া অবশ্য অ্যান্টিবডির মতো এত আলোচিত নয়। তবে সংক্রমণের বিরুদ্ধে লড়াই এবং দীর্ঘমেয়াদি সুরক্ষায় সমান গুরুত্বপূর্ণ ভূমিকা পালন করে। গবেষণাসংক্রান্ত নিবন্ধ প্রকাশিত হয়েছে ‘নেচার ইমিউনোলজি’ সাময়িকীতে। তাঁরা বলছেন, গবেষণার ক্ষেত্রে কোভিড-১৯ মৃদু সংক্রমণের শিকার ২৮ ব্যক্তির রক্তের নমুনা, ১৪ জন গুরুতর অসুস্থ ও ১৬ জন সুস্থ ব্যক্তির রক্তের নমুনা পরীক্ষা করেছেন। গবেষণা নিবন্ধে বলা হয়, সংক্রমিত ব্যক্তিদের ক্ষেত্রে টি-সেলের তীব্র প্রতিক্রিয়া তাঁরা দেখেছেন। এ ক্ষেত্রে মৃদু ও গুরুতর অসুস্থ ব্যক্তিদের ক্ষেত্রে প্রতিক্রিয়ার ভিন্নতা পাওয়া গেছে।"""

# Now you can use this in your inference code
print(sample_text)

‘ইকোনমিস্ট’ লিখেছে, অ্যান্টিবডির চার মাস স্থায়ী হওয়ার খবরটি দুই কারণে আনন্দের। অ্যান্টিবডি যত দিন পর্যন্ত শরীরে টিকবে, তত দিন সংক্রমণ থেকে সুরক্ষিত থাকা সম্ভব। অর্থাৎ, এমন এক টিকার প্রয়োজন হবে, যা অ্যান্টিবডির উত্পাদনকে প্ররোচিত করতে পারে এবং দীর্ঘস্থায়ী সুরক্ষা দিতে পারে। এগুলো খুঁজে বের করাও সহজ। এটি আভাস দেয়, ব্যাপক হারে অ্যান্টিবডি শনাক্তকরণ ফলাফল মোটামুটি নির্ভুল হওয়া উচিত। দ্বিতীয় আরেকটি গবেষণার নেতৃত্ব দিয়েছেন যুক্তরাজ্যের মেডিকেল রিসার্চ কাউন্সিলের (এমআরসি) ইমিউনোলজিস্ট তাও দং। তিনি টি-সেল শনাক্তকরণে কাজ করেছেন। টি-সেল শনাক্তকরণের প্রক্রিয়া অবশ্য অ্যান্টিবডির মতো এত আলোচিত নয়। তবে সংক্রমণের বিরুদ্ধে লড়াই এবং দীর্ঘমেয়াদি সুরক্ষায় সমান গুরুত্বপূর্ণ ভূমিকা পালন করে। গবেষণাসংক্রান্ত নিবন্ধ প্রকাশিত হয়েছে ‘নেচার ইমিউনোলজি’ সাময়িকীতে। তাঁরা বলছেন, গবেষণার ক্ষেত্রে কোভিড-১৯ মৃদু সংক্রমণের শিকার ২৮ ব্যক্তির রক্তের নমুনা, ১৪ জন গুরুতর অসুস্থ ও ১৬ জন সুস্থ ব্যক্তির রক্তের নমুনা পরীক্ষা করেছেন। গবেষণা নিবন্ধে বলা হয়, সংক্রমিত ব্যক্তিদের ক্ষেত্রে টি-সেলের তীব্র প্রতিক্রিয়া তাঁরা দেখেছেন। এ

In [ ]:
# Ensure the model is on CPU
model.to("cpu")

# Make sure the input_ids tensor is on the same device as the model (CPU in this case)
device = model.device  # This should now be 'cpu'
input_ids = tokenizer.encode(sample_text, return_tensors="pt", max_length=1024, truncation=True).to(device)

# Generate summary
outputs = model.generate(input_ids, max_length=200, num_beams=7, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

বাংলাদেশে কোভিড-১৯ মৃদু সংক্রমণের শিকার ২৮ ব্যক্তির রক্তের নমুনা পরীক্ষা করেছেন।


In [ ]:
save_path = '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/'

# Make all model parameters contiguous
for param in model.parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

# Save the model and tokenizer
model.save_pretrained(f"{save_path}fine_tuned_mt5_summarization_v3")
tokenizer.save_pretrained(f"{save_path}fine_tuned_mt5_summarization_v3")

('/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization_v3/tokenizer_config.json',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization_v3/special_tokens_map.json',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization_v3/spiece.model',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization_v3/added_tokens.json')

In [ ]:
from google.colab import runtime
runtime.unassign()